In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from torch.autograd import Variable

In [2]:
input_size = 10 # input dimension (word embedding) D
hidden_size = 30 # hidden dimension H
batch_size = 3
length = 4



In [3]:
rnn = nn.RNN(input_size, hidden_size,num_layers=1,bias=False,nonlinearity='tanh', batch_first=True, dropout=0, bidirectional=False)



30

In [6]:
hidden.shape

torch.Size([1, 3, 30])

In [4]:
input_ = Variable(torch.ones(batch_size,length,input_size)) # B,T,D  <= batch_first
hidden = Variable(torch.zeros(1,batch_size,hidden_size)) # 1,B,H    (num_layers * num_directions, batch, hidden_size)

In [9]:
output, hidden_2 = rnn(input_,hidden)

In [10]:
output.shape

torch.Size([3, 4, 30])

In [37]:
hidden.shape

torch.Size([1, 3, 30])

In [ ]:


output, hidden = rnn(input_) 

# (배치 사이즈, 시퀀스 길이, input 차원)을 가지는 Input 
# (1,배치 사이즈, hidden 차원)을 가지는 초기 hidden state

In [47]:
output.size()

torch.Size([3, 4, 30])

In [44]:
output.size()

torch.Size([3, 4, 30])

In [48]:
hidden.size()

torch.Size([1, 3, 30])

In [42]:
hidden.size()

torch.Size([2, 3, 30])

In [34]:
output

tensor([[[-0.2400,  0.4409, -0.6815,  0.3815,  0.4817,  0.3543,  0.0278,
          -0.2422, -0.5285,  0.2401, -0.5547, -0.2543, -0.5245, -0.4616,
          -0.1528,  0.2695,  0.4874, -0.3386,  0.4287, -0.0242, -0.0146,
          -0.2488,  0.4442,  0.3694,  0.2878,  0.3819,  0.3784,  0.2036,
           0.3754,  0.4077],
         [-0.1207,  0.1482, -0.6327,  0.3872,  0.5229,  0.3736, -0.3267,
          -0.2428, -0.2329,  0.2017, -0.6798,  0.2052, -0.5195, -0.5068,
           0.1413,  0.5225,  0.5724, -0.3839,  0.3542, -0.4187,  0.0185,
          -0.0204,  0.5256,  0.3115,  0.4012, -0.0352,  0.4341,  0.3334,
           0.3463,  0.5739],
         [-0.1083,  0.2041, -0.6793,  0.5248,  0.6526,  0.4556, -0.3271,
          -0.1001, -0.2185,  0.1026, -0.7892,  0.2265, -0.3555, -0.5174,
           0.0561,  0.5115,  0.4459, -0.4866,  0.4238, -0.2722,  0.0156,
          -0.1135,  0.4092,  0.2613,  0.4166,  0.0428,  0.3991,  0.3373,
           0.4260,  0.5506],
         [-0.0596,  0.2062, -0.6812, 

In [35]:
hidden

tensor([[[-0.0596,  0.2062, -0.6812,  0.5150,  0.6734,  0.4206, -0.3502,
          -0.1237, -0.2323,  0.0551, -0.7724,  0.1610, -0.3251, -0.4898,
           0.1142,  0.4912,  0.3822, -0.4616,  0.4632, -0.2536,  0.0102,
          -0.1147,  0.4144,  0.3260,  0.5088,  0.0060,  0.3943,  0.3139,
           0.3733,  0.5697],
         [-0.0596,  0.2062, -0.6812,  0.5150,  0.6734,  0.4206, -0.3502,
          -0.1237, -0.2323,  0.0551, -0.7724,  0.1610, -0.3251, -0.4898,
           0.1142,  0.4912,  0.3822, -0.4616,  0.4632, -0.2536,  0.0102,
          -0.1147,  0.4144,  0.3260,  0.5088,  0.0060,  0.3943,  0.3139,
           0.3733,  0.5697],
         [-0.0596,  0.2062, -0.6812,  0.5150,  0.6734,  0.4206, -0.3502,
          -0.1237, -0.2323,  0.0551, -0.7724,  0.1610, -0.3251, -0.4898,
           0.1142,  0.4912,  0.3822, -0.4616,  0.4632, -0.2536,  0.0102,
          -0.1147,  0.4144,  0.3260,  0.5088,  0.0060,  0.3943,  0.3139,
           0.3733,  0.5697]]], grad_fn=<StackBackward>)

In [ ]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
import time
import numpy as np



In [ ]:
#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
lmbda         = 0.95
eps_clip      = 0.1
K_epoch       = 2
T_horizon     = 20

In [ ]:
class PPO(nn.Module):
    def __init__(self):
        super(PPO, self).__init__()
        self.data = []
        
        self.fc1   = nn.Linear(4,64)
        self.lstm  = nn.LSTM(64,32)
        self.fc_pi = nn.Linear(32,2)
        self.fc_v  = nn.Linear(32,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self, x, hidden):
        x = F.relu(self.fc1(x))
        x = x.view(-1, 1, 64)
        x, lstm_hidden = self.lstm(x, hidden)
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=2)
        return prob, lstm_hidden
    
    def v(self, x, hidden):
        x = F.relu(self.fc1(x))
        x = x.view(-1, 1, 64)
        x, lstm_hidden = self.lstm(x, hidden)
        v = self.fc_v(x)
        return v
      
    def put_data(self, transition):
        self.data.append(transition)
        
    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, prob_a_lst, h_in_lst, h_out_lst, done_lst = [], [], [], [], [], [], [], []
        for transition in self.data:
            s, a, r, s_prime, prob_a, h_in, h_out, done = transition
            
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            prob_a_lst.append([prob_a])
            h_in_lst.append(h_in)
            h_out_lst.append(h_out)
            done_mask = 0 if done else 1
            done_lst.append([done_mask])
            
        s,a,r,s_prime,done_mask,prob_a = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
                                         torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
                                         torch.tensor(done_lst, dtype=torch.float), torch.tensor(prob_a_lst)
        self.data = []
        return s,a,r,s_prime, done_mask, prob_a, h_in_lst[0], h_out_lst[0]
        
    def train_net(self):
        s,a,r,s_prime,done_mask, prob_a, (h1_in, h2_in), (h1_out, h2_out) = self.make_batch()
        first_hidden  = (h1_in.detach(), h2_in.detach())
        second_hidden = (h1_out.detach(), h2_out.detach())

        for i in range(K_epoch):
            v_prime = self.v(s_prime, second_hidden).squeeze(1)
            td_target = r + gamma * v_prime * done_mask
            v_s = self.v(s, first_hidden).squeeze(1)
            delta = td_target - v_s
            delta = delta.detach().numpy()
            
            advantage_lst = []
            advantage = 0.0
            for item in delta[::-1]:
                advantage = gamma * lmbda * advantage + item[0]
                advantage_lst.append([advantage])
            advantage_lst.reverse()
            advantage = torch.tensor(advantage_lst, dtype=torch.float)

            pi, _ = self.pi(s, first_hidden)
            pi_a = pi.squeeze(1).gather(1,a)
            ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))  # a/b == log(exp(a)-exp(b))

            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
            loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(v_s, td_target.detach())

            self.optimizer.zero_grad()
            loss.mean().backward(retain_graph=True)
            self.optimizer.step()
        


In [ ]:

env = gym.make('CartPole-v1')
model = PPO()
score = 0.0
print_interval = 20

for n_epi in range(10000):
    h_out = (torch.zeros([1, 1, 32], dtype=torch.float), torch.zeros([1, 1, 32], dtype=torch.float))
    s = env.reset()
    done = False
        
    while not done:
        for t in range(T_horizon):
            h_in = h_out
            prob, h_out = model.pi(torch.from_numpy(s).float(), h_in)
            prob = prob.view(-1)
            m = Categorical(prob)
            a = m.sample().item()
            s_prime, r, done, info = env.step(a)

            model.put_data((s, a, r/100.0, s_prime, prob[a].item(), h_in, h_out, done))
            s = s_prime

            score += r
            if done:
                break
                    
        model.train_net()

    if n_epi%print_interval==0 and n_epi!=0:
        print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
        score = 0.0

env.close()



In [19]:
class PPO(nn.Module):
    def __init__(self):
        super(PPO, self).__init__()
        self.data = []
        
        self.fc1   = nn.Linear(4,64)
        self.lstm  = nn.LSTM(64,32)
        self.fc_pi = nn.Linear(32,4)
        self.fc_v  = nn.Linear(32,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self, x, hidden):
        x = F.relu(self.fc1(x))
        x = x.view(-1, 1, 64)
        x, lstm_hidden = self.lstm(x, hidden)
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=2)
        return prob, lstm_hidden
    
    def v(self, x, hidden):
        x = F.relu(self.fc1(x))
        x = x.view(-1, 1, 64)
        x, lstm_hidden = self.lstm(x, hidden)
        v = self.fc_v(x)
        return v
      
    def put_data(self, transition):
        self.data.append(transition)
        
    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, prob_a_lst, h_in_lst, h_out_lst, done_lst = [], [], [], [], [], [], [], []
        for transition in self.data:
            s, a, r, s_prime, prob_a, h_in, h_out, done = transition
            
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            prob_a_lst.append([prob_a])
            h_in_lst.append(h_in)
            h_out_lst.append(h_out)
            done_mask = 0 if done else 1
            done_lst.append([done_mask])
            
        s,a,r,s_prime,done_mask,prob_a = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
                                         torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
                                         torch.tensor(done_lst, dtype=torch.float), torch.tensor(prob_a_lst)
        self.data = []
        return s,a,r,s_prime, done_mask, prob_a, h_in_lst[0], h_out_lst[0]
        
    def train_net(self):
        s,a,r,s_prime,done_mask, prob_a, (h1_in, h2_in), (h1_out, h2_out) = self.make_batch()
        first_hidden  = (h1_in.detach(), h2_in.detach())
        second_hidden = (h1_out.detach(), h2_out.detach())

        for i in range(K_epoch):
            v_prime = self.v(s_prime, second_hidden).squeeze(1)
            td_target = r + gamma * v_prime * done_mask
            v_s = self.v(s, first_hidden).squeeze(1)
            delta = td_target - v_s
            delta = delta.detach().numpy()
            
            advantage_lst = []
            advantage = 0.0
            for item in delta[::-1]:
                advantage = gamma * lmbda * advantage + item[0]
                advantage_lst.append([advantage])
            advantage_lst.reverse()
            advantage = torch.tensor(advantage_lst, dtype=torch.float)

            pi, _ = self.pi(s, first_hidden)
            pi_a = pi.squeeze(1).gather(1,a)
            ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))  # a/b == log(exp(a)-exp(b))

            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
            loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(v_s, td_target.detach())

            self.optimizer.zero_grad()
            loss.mean().backward(retain_graph=True)
            self.optimizer.step()

In [22]:
a = PPO()
env = gym.make('CartPole-v1')